In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

def obter_situacao_resultado_com_login(nome_aluno, usuario, senha):
    caminho_driver = r"C:\Users\daniel.santos\Desktop\automaçaopx\chromedriver-win64\chromedriver-win64\chromedriver.exe"

    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(caminho_driver), options=options)
    wait = WebDriverWait(driver, 15)

    try:
        # 1. Acessa a página de login
        driver.get("https://mentorweb.pedreira.org/ceap/inicial.do?pcaes=0f595d7f836bd3dc4a7b90184b310a25")
        time.sleep(2)

        # 2. Preenche login e senha
        campo_login = wait.until(EC.element_to_be_clickable((By.ID, "j_username")))
        campo_login.clear()
        campo_login.send_keys(usuario)

        campo_senha = wait.until(EC.element_to_be_clickable((By.ID, "senha")))
        campo_senha.clear()
        campo_senha.send_keys(senha)
        campo_senha.send_keys(Keys.ENTER)
        time.sleep(3)

        # 3. Envia tecla F2 para abrir o menu "Atalhos"
        ActionChains(driver).send_keys(Keys.F2).perform()
        time.sleep(2)

        # 4. Aguarda o campo para pesquisar nome do aluno
        campo_nome = wait.until(EC.presence_of_element_located((By.ID, "opcaoAtalhoCampoNome")))
        campo_nome.clear()
        campo_nome.send_keys(nome_aluno)
        time.sleep(1)
        campo_nome.send_keys(Keys.ENTER)
        time.sleep(2)

        # 5. Clica no link "Períodos de matrícula"
        link_periodo = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Períodos de matrícula')]")))
        link_periodo.click()
        time.sleep(2)

        # 6. Espera e coleta as matrículas
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//tr[contains(@onclick, 'matriculaturma.do')]")))
        linhas = driver.find_elements(By.XPATH, "//tr[contains(@onclick, 'matriculaturma.do')]")
        matriculas = []
        for linha in linhas:
            periodo = linha.find_element(By.XPATH, "./td[1]").text.strip()
            onclick = linha.get_attribute("onclick")
            if "redirect('" in onclick:
                url = onclick.split("redirect('")[1].split("'")[0]
                matriculas.append((periodo, url))

        if not matriculas:
            print("⚠️ Nenhuma matrícula encontrada.")
            return None

        # 7. Seleciona a matrícula mais recente
        matriculas.sort(key=lambda x: x[0], reverse=True)
        url_mais_recente = matriculas[0][1]
        driver.get("https://mentorweb.pedreira.org" + url_mais_recente)
        time.sleep(2)

        # 8. Clica no link "Resultado nas fases"
        link_resultado = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//a[contains(@onclick, 'ingressoresultadofaseman.do')]")
        ))
        onclick = link_resultado.get_attribute("onclick")
        url_resultado = onclick.split("redirect('")[1].split("'")[0]
        driver.get("https://mentorweb.pedreira.org" + url_resultado)
        time.sleep(2)

        # 9. Coleta todas as fases e suas informações
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.delimitador")))
        linhas_fase = driver.find_elements(By.CSS_SELECTOR, "table.delimitador tbody tr")
        dados_resultado = []

        for linha in linhas_fase:
            try:
                select_periodo = Select(linha.find_element(By.CSS_SELECTOR, "select[name^='resultadoFase'][name$='periodoLetivoAux']"))
                periodo_texto = select_periodo.first_selected_option.text.strip()

                input_fase = linha.find_element(By.CSS_SELECTOR, "input[name^='resultadoFase'][name$='faseAux']")
                fase_valor = input_fase.get_attribute("value").strip()

                select_situacao = Select(linha.find_element(By.CSS_SELECTOR, "select[name^='resultadoFase'][name$='situacaoResultadoAux']"))
                situacao_texto = select_situacao.first_selected_option.text.strip()

                dados_resultado.append({
                    "periodo_letivo": periodo_texto,
                    "fase": fase_valor,
                    "situacao_resultado": situacao_texto
                })
            except:
                continue

        return dados_resultado

    except Exception as e:
        print("⚠️ Erro durante automação:", e)
        return None

    finally:
        driver.quit()


def main():
    nome_aluno = "Daniel dias tome dos santos"
    usuario = "seu_login"
    senha = "sua_senha"

    print(f"\n🔍 Consultando aluno: {nome_aluno}")
    dados = obter_situacao_resultado_com_login(nome_aluno, usuario, senha)

    if dados:
        print(f"\n📄 Resultados encontrados para {nome_aluno}:")
        for item in dados:
            print(f"  • Período: {item['periodo_letivo']} | Fase: {item['fase']} | Situação: {item['situacao_resultado']}")
    else:
        print(f"\n⚠️ Nenhum dado retornado para {nome_aluno}.")


if __name__ == "__main__":
    main()



🔍 Consultando aluno: {nome}

📄 Resultados encontrados para {nome}:
  • Período: 2024.2 | Fase: 4 | Situação: Concluído


In [38]:
import pandas as pd
from openpyxl import load_workbook

def atualizar_planilha_para_abas(caminho_arquivo, abas, usuario, senha):
    wb = load_workbook(caminho_arquivo)

    for aba in abas:
        print(f"\n📘 Processando aba: {aba}")
        ws = wb[aba]
        df = pd.read_excel(caminho_arquivo, sheet_name=aba)

       # Garante que as colunas existam e sejam string (para evitar FutureWarning)
        for coluna in ['Período', 'Fase', 'Situação']:
            if coluna not in df.columns:
                df[coluna] = ""
            df[coluna] = df[coluna].astype(str)
        
        for i, row in df.iterrows():
            nome = str(row['Nome']).strip()
        
            if not nome or nome.lower() in ["nan", ""]:
                continue
        
            print(f"\n🔎 Buscando dados para: {nome}")
            
            try:
                resultados = obter_situacao_resultado_com_login(nome, usuario, senha)
        
                if resultados:
                    mais_recente = sorted(resultados, key=lambda x: x["periodo_letivo"], reverse=True)[0]
        
                    df.at[i, 'Período'] = str(mais_recente['periodo_letivo'])
                    df.at[i, 'Fase'] = str(mais_recente['fase'])
                    df.at[i, 'Situação'] = str(mais_recente['situacao_resultado'])
                else:
                    df.at[i, 'Situação'] = "NÃO ENCONTRADO"
        
            except Exception as e:
                print(f"⚠️ Erro ao processar {nome}: {e}")
                df.at[i, 'Situação'] = "ERRO NA BUSCA"
                continue
        
            linha_excel = i + 2
            ws[f"C{linha_excel}"] = df.at[i, 'Período']
            ws[f"D{linha_excel}"] = df.at[i, 'Fase']
            ws[f"E{linha_excel}"] = df.at[i, 'Situação']



    # Salva tudo
    wb.save(caminho_arquivo)
    print("\n✅ Todas as abas foram processadas com sucesso.")


In [39]:
# Exemplo de uso:
usuario = "seu_login"
senha = "sua_senha"
arquivo = "Seu_Arquivo.xlsx"
aba = "Aba_aser_editada"

processar_planilha_preservando_formatacao(arquivo, aba)


🔄 1/3 - {nome} processado.
🔄 2/3 - {nome} processado.
🔄 3/3 - {nome} processado.

✅ Atualização concluída com sucesso.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

def obter_situacao_resultado_com_login(nome_aluno, usuario, senha):
    caminho_driver = r"C:\Users\daniel.santos\Desktop\automaçaopx\chromedriver-win64\chromedriver-win64\chromedriver.exe"

    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(caminho_driver), options=options)
    wait = WebDriverWait(driver, 15)

    try:
        # 1. Acessa a página de login
        driver.get("https://mentorweb.pedreira.org/ceap/inicial.do?pcaes=0f595d7f836bd3dc4a7b90184b310a25")
        time.sleep(2)

        # 2. Preenche login e senha
        campo_login = wait.until(EC.element_to_be_clickable((By.ID, "j_username")))
        campo_login.clear()
        campo_login.send_keys(usuario)

        campo_senha = wait.until(EC.element_to_be_clickable((By.ID, "senha")))
        campo_senha.clear()
        campo_senha.send_keys(senha)
        campo_senha.send_keys(Keys.ENTER)
        time.sleep(3)

        # 3. Envia tecla F2 para abrir o menu "Atalhos"
        ActionChains(driver).send_keys(Keys.F2).perform()
        time.sleep(2)

        # 4. Aguarda o campo para pesquisar nome do aluno
        campo_nome = wait.until(EC.presence_of_element_located((By.ID, "opcaoAtalhoCampoNome")))
        campo_nome.clear()
        campo_nome.send_keys(nome_aluno)
        time.sleep(1)
        campo_nome.send_keys(Keys.ENTER)
        time.sleep(2)

        # 5. Clica no link "Períodos de matrícula"
        link_periodo = wait.until(EC.element_to_be_clickable((By.XPATH, "//a[contains(text(), 'Períodos de matrícula')]")))
        link_periodo.click()
        time.sleep(2)

        # 6. Espera e coleta as matrículas
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//tr[contains(@onclick, 'matriculaturma.do')]")))
        linhas = driver.find_elements(By.XPATH, "//tr[contains(@onclick, 'matriculaturma.do')]")
        matriculas = []
        for linha in linhas:
            periodo = linha.find_element(By.XPATH, "./td[1]").text.strip()
            onclick = linha.get_attribute("onclick")
            if "redirect('" in onclick:
                url = onclick.split("redirect('")[1].split("'")[0]
                matriculas.append((periodo, url))

        if not matriculas:
            print("⚠️ Nenhuma matrícula encontrada.")
            return None

        # 7. Seleciona a matrícula mais recente
        matriculas.sort(key=lambda x: x[0], reverse=True)
        url_mais_recente = matriculas[0][1]
        driver.get("https://mentorweb.pedreira.org" + url_mais_recente)
        time.sleep(2)

                # 8. Coleta o nome da Turma
        try:
            elemento_turma = wait.until(EC.presence_of_element_located((
                By.XPATH,
                "//td[contains(text(), 'Turma')]/following-sibling::td/a"
            )))
            nome_turma = elemento_turma.text.strip()
        except Exception as e:
            print("⚠️ Erro ao coletar a Turma:", e)
            nome_turma = "TURMA NÃO ENCONTRADA"


        # 8. Clica no link "Resultado nas fases"
        link_resultado = wait.until(EC.presence_of_element_located(
            (By.XPATH, "//a[contains(@onclick, 'ingressoresultadofaseman.do')]")
        ))
        onclick = link_resultado.get_attribute("onclick")
        url_resultado = onclick.split("redirect('")[1].split("'")[0]
        driver.get("https://mentorweb.pedreira.org" + url_resultado)
        time.sleep(2)

        # 9. Coleta todas as fases e suas informações
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "table.delimitador")))
        linhas_fase = driver.find_elements(By.CSS_SELECTOR, "table.delimitador tbody tr")
        dados_resultado = []

        for linha in linhas_fase:
            try:
                select_periodo = Select(linha.find_element(By.CSS_SELECTOR, "select[name^='resultadoFase'][name$='periodoLetivoAux']"))
                periodo_texto = select_periodo.first_selected_option.text.strip()

                input_fase = linha.find_element(By.CSS_SELECTOR, "input[name^='resultadoFase'][name$='faseAux']")
                fase_valor = input_fase.get_attribute("value").strip()

                select_situacao = Select(linha.find_element(By.CSS_SELECTOR, "select[name^='resultadoFase'][name$='situacaoResultadoAux']"))
                situacao_texto = select_situacao.first_selected_option.text.strip()

                dados_resultado.append({
                    "periodo_letivo": periodo_texto,
                    "fase": fase_valor,
                    "situacao_resultado": situacao_texto
                })
            except:
                continue

        return dados_resultado

    except Exception as e:
        print("⚠️ Erro durante automação:", e)
        return None

    finally:
        driver.quit()


def main():
    nome_aluno = "{nome}"
    usuario = "seu_login"
    senha = "sua_senha"

    print(f"\n🔍 Consultando aluno: {nome_aluno}")
    resultado = obter_situacao_resultado_com_login(nome_aluno, usuario, senha)

    if resultado:
        turma = resultado['turma']
        fases = resultado['fases']

        print(f"\n🏷️ Turma: {turma}")
        print(f"\n📄 Resultados encontrados para {nome_aluno}:")

        for item in fases:
            print(f"  • Período: {item['periodo_letivo']} | Fase: {item['fase']} | Situação: {item['situacao_resultado']}")

    else:
        print(f"\n⚠️ Nenhum dado retornado para {nome_aluno}.")


if __name__ == "__main__":
    main()
